In [ ]:
import os

if os.getcwd().endswith("notebooks"):
    os.chdir("..")
print("cwd:", os.getcwd())

In [1]:
from matplotlib import pyplot as plt
import numpy as np

from rover import VoxelGrid

In [3]:
grid = VoxelGrid.from_npz("arrow/results/wiselab2/demo/map.npz", decimate=2)

In [ ]:
grid_crop = grid.crop(left=(100, 100, 30), right=(310, 450, 120))

fig, axs = plt.subplots(1, 1, figsize=(12, 4))
axs.imshow(np.sum(grid_crop.data, axis=2).T)

In [ ]:
from jax.scipy.signal import convolve

grid_cfar = grid_crop.cfar(guard_band=1, window_size=3, convolve_func=convolve)
grid_clip = grid_crop.normalize(left=10.0, right=99.5)

layers = [(3.0, 0.25), (2.0, 0.2), (1.5, 0.15)]
masks = []

mask = False
for cfar, sigma in layers:
    layer = (grid_clip.data > sigma) & (grid_crop.data > cfar * grid_cfar.data)
    masks.append(layer & ~mask)
    mask |= layer


fig, axs = plt.subplots(1, 3, figsize=(16, 8))

for ax, m in zip(axs, masks):
    ax.imshow(m[:, :, 30])

In [9]:
grid_clip.as_pcd("arrow/results/wiselab2/demo1.pcd", masks[0])
grid_clip.as_pcd("arrow/results/wiselab2/demo2.pcd", masks[1])
grid_clip.as_pcd("arrow/results/wiselab2/demo3.pcd", masks[2])